In [1]:
[Similar code structure as Sierra Leone, but with:
df = pd.read_csv('../data/togo-dapaong_qc.csv')
and appropriate title changes
]

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Togo Solar Data Analysis\n',
    '\n',
    '## Data Loading and Cleaning']},
  {'cell_type': 'code',
   'execution_count': 0,
   'metadata': {},
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'from scipy import stats\n',
    'from utils import load_and_clean_data, detect_outliers, plot_time_series, create_wind_rose\n',
    '\n',
    '# Set style\n',
    "plt.style.use('seaborn')\n",
    "sns.set_palette('husl')"]},
  {'cell_type': 'code',
   'execution_count': 0,
   'metadata': {},
   'source': ['# Load data\n',
    "df = pd.read_csv('../data/togo-dapaong_qc.csv')\n",
    "df['Timestamp'] = pd.to_datetime(df['Timestamp'])\n",
    'print(f"Data shape: {df.shape}")\n',
    'df.head()']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## Data Cleaning and Preprocessing']},
  {'cell_type': 'code',
   'executi